In [1]:
import pandas as pd

import json
import glob
import os
import re

In [2]:
base = "../exp/"
paths = glob.glob(base + "applications/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
algos = ['Virtual Topocore Component Query', 'Virtual Topocore Bidirectional Core Query']
queries = pd.DataFrame.from_records([{ 
    **algo,
    'exp': exp['experiment'], 
    'graph': run['args'][1],
    'num_queue_pops': algo.get('num_queue_pops', 0) + algo.get('core_search', {}).get('num_queue_pops', 0),
    'num_queue_pushs': algo.get('num_queue_pushs', 0) + algo.get('core_search', {}).get('num_queue_pushs', 0),
    'num_relaxed_arcs': algo.get('num_relaxed_arcs', 0) + algo.get('core_search', {}).get('num_relaxed_arcs', 0)
    } for run in data for exp in run.get('experiments', []) for algo in exp['algo_runs'] if algo.get('algo') in algos])

queries = queries.append(pd.DataFrame.from_records([{ 
    **algo,
    'exp': run['program'], 
    'graph': run['args'][1],
    'num_queue_pops': algo.get('num_queue_pops', 0) + algo.get('core_search', {}).get('num_queue_pops', 0),
    'num_queue_pushs': algo.get('num_queue_pushs', 0) + algo.get('core_search', {}).get('num_queue_pushs', 0),
    'num_relaxed_arcs': algo.get('num_relaxed_arcs', 0) + algo.get('core_search', {}).get('num_relaxed_arcs', 0)
    } for run in data for algo in run.get('algo_runs', []) if algo.get('algo') in algos]))

In [4]:
algos = ['Dijkstra Query', 'Bidrectional Dijkstra Query']
dijkstra_queries = pd.DataFrame.from_records([{ 
    'exp': exp['experiment'], 
    'graph': run['args'][1],
    **algo } 
    for run in data for exp in run.get('experiments', []) for algo in exp['algo_runs'] if algo.get('algo') in algos])

dijkstra_queries = dijkstra_queries.append(pd.DataFrame.from_records([{ 
    'exp': run['program'], 
    'graph': run['args'][1],
    **algo } 
    for run in data for algo in run.get('algo_runs', []) if algo.get('algo') in algos]))

In [5]:
queries['affected'] = queries['lower_bound'] != queries['result']
queries['increase'] = (queries['result'] / queries['lower_bound'] - 1) * 100

In [7]:
queries.groupby(['graph', 'exp']).count()

algo  \
graph                                           exp                          
/home/i11/zeitz/projects/chpot/data/europe/     perfect              10000   
/home/i11/zeitz/projects/chpot/data/ger06/      chpot_td             10000   
/home/i11/zeitz/projects/chpot/data/osm_ger/    bidir_chpot_live     10000   
                                                bidir_chpot_turns    10000   
                                                cchpot_live          10000   
                                                cchpot_turns          9041   
                                                chpot_live           10000   
                                                chpot_turns           9041   
                                                no_highways          20000   
                                                no_tunnels           20000   
                                                perfect              10000   
/home/i11/zeitz/projects/chpot/data/osm_ger_td/ chpot_td             10000   
                                                chpot_td_live        10000   
                                                chpot_turns_td_live   9069   
/home/i11/zeitz/projects/chpot/data/ptv17/      chpot_td             10000   
/home/i11/zeitz/projects/chpot/data/ptv20/      chpot_td             10000   

                                                                     core_search  \
graph                                           exp                                
/home/i11/zeitz/projects/chpot/data/europe/     perfect                    10000   
/home/i11/zeitz/projects/chpot/data/ger06/      chpot_td                   10000   
/home/i11/zeitz/projects/chpot/data/osm_ger/    bidir_chpot_live               0   
                                                bidir_chpot_turns              0   
                                                cchpot_live                10000   
                                                cchpot_turns                8951   
                                                chpot_live                 10000   
                                                chpot_turns                 8951   
                                                no_highways                 9968   
                                                no_tunnels                  9993   
                                                perfect                    10000   
/home/i11/zeitz/projects/chpot/data/osm_ger_td/ chpot_td                   10000   
                                                chpot_td_live              10000   
                                                chpot_turns_td_live         8979   
/home/i11/zeitz/projects/chpot/data/ptv17/      chpot_td                   10000   
/home/i11/zeitz/projects/chpot/data/ptv20/      chpot_td                    9999   

                                                                      from  \
graph                                           exp                          
/home/i11/zeitz/projects/chpot/data/europe/     perfect              10000   
/home/i11/zeitz/projects/chpot/data/ger06/      chpot_td             10000   
/home/i11/zeitz/projects/chpot/data/osm_ger/    bidir_chpot_live     10000   
                                                bidir_chpot_turns    10000   
                                                cchpot_live          10000   
                                                cchpot_turns          9041   
                                                chpot_live           10000   
                                                chpot_turns           9041   
                                                no_highways          20000   
                                                no_tunnels           20000   
                                                perfect              10000   
/home/i11/zeitz/projects/chpot/data/osm_ger_td/ chpot_td             10000   
                                                chpot_td_liv

In [13]:
queries.groupby(['graph', 'exp']).mean()[['running_time_ms', 'num_queue_pushs', 'increase']]

running_time_ms  \
graph                                           exp                                    
/home/i11/zeitz/projects/chpot/data/europe/     perfect                     0.859499   
/home/i11/zeitz/projects/chpot/data/ger06/      chpot_td                    4.156492   
/home/i11/zeitz/projects/chpot/data/osm_ger/    bidir_chpot_live          191.115389   
                                                bidir_chpot_turns           1.119111   
                                                cchpot_live                 1.524210   
                                                cchpot_turns                6.349214   
                                                chpot_live                127.521422   
                                                chpot_turns                 4.360090   
                                                no_highways               402.564449   
                                                no_tunnels                 31.756817   
                                                perfect                     0.687186   
/home/i11/zeitz/projects/chpot/data/osm_ger_td/ chpot_td                  122.408242   
                                                chpot_td_live             202.969159   
                                                chpot_turns_td_live       538.389044   
/home/i11/zeitz/projects/chpot/data/ptv17/      chpot_td                   81.072491   
/home/i11/zeitz/projects/chpot/data/ptv20/      chpot_td                   95.380791   

                                                                     num_queue_pushs  \
graph                                           exp                                    
/home/i11/zeitz/projects/chpot/data/europe/     perfect                  1063.473700   
/home/i11/zeitz/projects/chpot/data/ger06/      chpot_td                 6386.854500   
/home/i11/zeitz/projects/chpot/data/osm_ger/    bidir_chpot_live       186422.318500   
                                                bidir_chpot_turns         760.298400   
                                                cchpot_live              1497.233000   
                                                cchpot_turns            11538.657892   
                                                chpot_live             193240.168200   
                                                chpot_turns              8083.888176   
                                                no_highways            528154.268450   
                                                no_tunnels              41481.007400   
                                                perfect                   664.876700   
/home/i11/zeitz/projects/chpot/data/osm_ger_td/ chpot_td               103454.306300   
                                                chpot_td_live          168168.084700   
                                                chpot_turns_td_live    725315.877826   
/home/i11/zeitz/projects/chpot/data/ptv17/      chpot_td                79813.405300   
/home/i11/zeitz/projects/chpot/data/ptv20/      chpot_td                72821.817800   

                                                                      increase  
graph                                           exp                             
/home/i11/zeitz/projects/chpot/data/europe/     perfect               0.000000  
/home/i11/zeitz/projects/chpot/data/ger06/      chpot_td              3.056231  
/home/i11/zeitz/projects/chpot/data/osm_ger/    bidir_chpot_live     15.311185  
                                                bidir_chpot_turns     1.130937  
                                                cchpot_live           0.251448  
                                                cchpot_turns          1.029940  
                                                chpot_live           15.311786  
                                                chpot_turns           1.109166  
                                                no_highways          43.325570  
        

In [14]:
queries.query('exp == "cchpot_live"')['increase'].describe()

count    10000.000000
mean         0.251448
std          0.462309
min          0.000000
25%          0.000000
50%          0.099557
75%          0.328084
max         11.284656
Name: increase, dtype: float64

In [7]:
table = queries.groupby(['graph', 'exp']).agg(
    running_time_ms=('running_time_ms', 'mean'), 
    num_pot_evals=('num_pot_evals', 'mean'), 
    num_relaxed_arcs=('num_relaxed_arcs', 'mean'),
    increase=('increase', 'mean'),
    affected=('affected', 'sum'),
    size=('affected', 'count'))
table['affected'] = table['affected'] * 100 / table['size']
table = table[table.columns[0:-1]]
table['num_relaxed_arcs'] = table['num_relaxed_arcs'] / 1000
table['num_pot_evals'] = table['num_pot_evals'] / 1000
table = table.join(preprocessing.groupby('graph').mean())
table = table.join(dijkstra_queries.groupby(['graph', 'exp']).agg(dijkstra_running_time_ms=('running_time_ms', 'mean')))
#table = table.round(1)
print(table.to_latex())
table

\begin{tabular}{llrrrrrrr}
\toprule
                                         &            &  running\_time\_ms &  num\_pot\_evals &  num\_relaxed\_arcs &  increase &  affected &  preprocessing\_running\_time\_s &  dijkstra\_running\_time\_ms \\
graph & exp &                  &                &                   &           &           &                               &                           \\
\midrule
/algoDaten/graphs/cleaned\_td\_road\_data/de/day/dido/ & chpot\_td &         4.341473 &       7.621476 &         30.077662 &  1.030244 &     98.23 &                     59.086588 &                604.205920 \\
/algoDaten/graphs/cleaned\_td\_road\_data/ptv17-eur-car/day/di/ & chpot\_td &        85.371324 &     103.096165 &        500.442628 &  1.038714 &     99.48 &                    292.520587 &               3245.881700 \\
/algoDaten/zeitz/roadgraphs/europe/ & fastest\_times\_10 &        50.767913 &     111.816654 &        390.435148 &  1.028436 &     96.28 &                    273.

running_time_ms  \
graph                                              exp                                 
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td                 4.341473   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td                85.371324   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10        50.767913   
                                                   fastest_times_2          6.696532   
                                                   random_times_10          0.975721   
                                                   random_times_2           0.970561   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live             107.523830   
                                                   chpot_turns             38.234487   
                                                   no_highways            315.260351   
                                                   no_tunnels              29.906702   

                                                                     num_pot_evals  \
graph                                              exp                               
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td               7.621476   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td             103.096165   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10     111.816654   
                                                   fastest_times_2       13.683046   
                                                   random_times_10        1.127394   
                                                   random_times_2         1.111212   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live           218.899497   
                                                   chpot_turns           85.649110   
                                                   no_highways          661.362835   
                                                   no_tunnels            60.373990   

                                                                     num_relaxed_arcs  \
graph                                              exp                                  
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td                 30.077662   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td                500.442628   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10        390.435148   
                                                   fastest_times_2          42.021788   
                                                   random_times_10           2.908221   
                                                   random_times_2            2.864245   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live              801.182286   
                                                   chpot_turns             178.109398   
                                                   no_highways            2465.660474   
                                                   no_tunnels              224.124574   

                                                                     increase  \
graph                                              exp                          
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td          1.030244   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td          1.038714   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10  1.028436   
                                                   fastest_times_2   1.006781   
                                                   random_times_10   1.000056   
                                                   random_times_2    1.000007   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live        1.137614   
                                                   chpot_turns       1.058078   
                                                   no_highways       1.392460   
                   